In [1]:
!pip install dipy
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.2 MB/s eta 0:00:00
  Attempting uninstall: nibabel
    Found existing installation: nibabel 4.0.2
    Uninstalling nibabel-4.0.2:
      Successfully uninstalled nibabel-4.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 776.1 kB/s eta 0:00:00


In [2]:
import os
from google.colab import drive
mount_path = '/content/drive'
drive_path = mount_path+"/MyDrive/dti-transformer/code/cl-training"
test_data = mount_path+'/MyDrive/dti-transformer/dti_data'
drive.mount(mount_path)
os.chdir(drive_path)

Mounted at /content/drive


In [3]:
# Daniel Bandala @ nov-2022
# dti-model validation script
# general libraries
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import log10, sqrt
# diffussion image processing
from dipy.io.image import load_nifti
# import torch libraries
import torch
from torch import nn
# import dataset auxiliar libraries
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
#from unet_model import DiffusionTensorModelUNet
from data_loader_cl import data_preprocessing, normalize_data
#from other.unet import UNet
from torchvision.transforms import functional as TF
from timeit import default_timer as timer
import time
# import model
import sys, glob
sys.path.insert(1, '../model')
from dti_model import DiffusionTensorModel

In [4]:
results_path = drive_path+"/results/FA"

In [5]:
signals = 7
maps = ["FA"] #"MD","MO","L1","L2","L3",FA

In [22]:
# load model
checkpoint = torch.load(results_path+'/../dti_fa.weights') #dti_fa.weights FA_2023-03-30
model = DiffusionTensorModel(
    in_chans=signals,
    out_chans=1,
    img_size=140,
    embed_dim=64,
    n_heads=[1,2,4,8],
    mlp_ratio=[2,2,4,4],
    reduction_ratio=1,
    depth_prob=0.2,
    tanh_output=False
)
# use model in cpu for validation (gpu for training)
_ = model.to('cpu')
_ = model.load_state_dict(checkpoint) #torch.load(, map_location=torch.device('cpu'))
_ = model.eval()

In [ ]:
# load model
checkpoint = torch.load(results_path+'/unet/dti_rgb.weights') #dti_fa.weights FA_2023-03-30
model = DiffusionTensorModelUNet(
    in_chans=signals,
    out_chans=3,
    img_size=140,
    embed_dim=64,
    n_heads=[1,2,4,8],
    mlp_ratio=[2,2,4,4],
    reduction_ratio=1,
    depth_prob=0.2,
    tanh_output=False
)
# use model in cpu for validation (gpu for training)
_ = model.to('cpu')
_ = model.load_state_dict(checkpoint) #torch.load(, map_location=torch.device('cpu'))
_ = model.eval()

In [ ]:
# load unet model
weights = torch.load(results_path+'/unet/dti_rgb.weights')
model = UNet(
    enc_channels=(signals,64,128,256,512,1024),
    dec_channels=(1024,512,256,128,64),
    out_chans=3,
    img_size=140
)
# use model in cpu for validation (gpu for training)
_ = model.to('cpu')
_ = model.load_state_dict(weights) #torch.load(, map_location=torch.device('cpu'))
_ = model.eval()

In [ ]:
test_list = [test_data+'/HCP/test/case_12',
              test_data+'/HCP/test/case_14',
              test_data+'/HCP/test/case_31',
              test_data+'/ADNI/test/case_13',
              test_data+'/ADNI/test/case_14',
              test_data+'/ADNI/test/case_30'
             ]

test_list = [test_data+'/HCP/test/case_12',
              test_data+'/HCP/test/case_14',
              test_data+'/HCP/test/case_31'
             ]

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# copy model to gpu
_ = model.to(device)

In [23]:
# get cases list
path_list = glob.glob(os.path.join(test_data, 'INUTR', 'test', 'case_*')) #INUTR
path_list

['/content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_44',
 '/content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_15',
 '/content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_03',
 '/content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_34',
 '/content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_08',
 '/content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_28',
 '/content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_77',
 '/content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_04']

In [24]:
start = time.time()
test_results = [["Slice","MSE","NMSE","SSIM","PSNR","Full path"]]
for data_path in path_list:
    print(f"Processing {data_path}")
    data_eval, label_eval = data_preprocessing(data_path, maps=maps, signals=signals, dti_folder='output')
    for sidx in range(data_eval.shape[0]):
        data = data_eval[sidx]
        label = label_eval[sidx]
        with torch.no_grad():
            output = model(data)
        # detach data
        label_np = label.detach().numpy()
        output_np = output.detach().numpy()
        # calculate metrics
        label_mean = label_np.mean()
        mse = mean_squared_error(label_np, output_np)
        nmse = mse/label_mean if label_mean!=0 else 0
        ssi = ssim(label_np, output_np, data_range=label_np.max() - label_np.min()) #channel_axis = 0
        psnr = 20*log10(1/sqrt(mse))
        # append results
        test_results.append([os.path.basename(data_path)+f'_{sidx}',mse,nmse,ssi,psnr,data_path])
end = time.time()
elapsed_time = end - start

Processing /content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_44
Processing /content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_15
Processing /content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_03
Processing /content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_34
Processing /content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_08
Processing /content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_28
Processing /content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_77
Processing /content/drive/MyDrive/dti-transformer/dti_data/INUTR/test/case_04


In [ ]:
drive.mount(mount_path, True)
os.chdir(drive_path)

Mounted at /content/drive


In [25]:
# save results to csv file
pd.DataFrame(test_results).to_csv(results_path+"/test_INUTR_NORMAL_TRAIN.csv", index=False, header=False)

In [ ]:
elapsed_time/180

1.4011349969440037

In [ ]:
elapsed_time/3

188.0440494219462

In [ ]:
# read fsl data
output_fsl,_ = load_nifti(os.path.join(data_path,'fsl','DTI_RGB.nii.gz'))
output_fsl =  np.array(output_fsl.transpose(2,0,1),dtype=np.float32) #2,3,0,1
output_fsl = torch.tensor(output_fsl)
output_fsl = normalize_data(output_fsl)
output_fsl = TF.resize(output_fsl, (140, 140))
output_fsl = output_fsl.detach().numpy()